**Setup in Google Colab**

In [1]:
!pip install langchain langchain-community
!pip install faiss-cpu
!pip install pypdf python-docx
!pip install sentence-transformers
!pip install transformers
!pip install docx2txt


   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   -----------------

**Upload File in Colab**

In [23]:
from ipywidgets import FileUpload
from IPython.display import display

uploaded = FileUpload(accept='.pdf,.docx,.txt', multiple=False)
display(uploaded)

def handle_upload(change):
    if uploaded.value:
        # uploaded.value is a tuple → take first element
        file_info = list(uploaded.value.values())[0] if isinstance(uploaded.value, dict) else list(uploaded.value)[0]
        
        # Get filename (some versions use 'name', older ones used metadata.name)
        file_name = file_info.get('name') or file_info.get('metadata', {}).get('name')
        
        # Save file locally
        with open(file_name, "wb") as f:
            f.write(file_info['content'])
        
        print("Uploaded:", file_name)

        # Make it available as file_path (like Colab)
        global file_path
        file_path = file_name

uploaded.observe(handle_upload, names='value')


FileUpload(value=(), accept='.pdf,.docx,.txt', description='Upload')

In [3]:
pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


**Load & Split Document**

In [24]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Pick loader based on file type
if file_path.endswith(".pdf"):
    loader = PyPDFLoader(file_path)
elif file_path.endswith(".docx"):
    loader = Docx2txtLoader(file_path)
else:
    loader = TextLoader(file_path)

docs = loader.load()

# Split into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = splitter.split_documents(docs)

print(f"Total Chunks: {len(documents)}")


Total Chunks: 2


**Create Embeddings & Vector Store**

In [25]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Small + Fast embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(documents, embeddings)


**Load an Open-Source LLM**

In [26]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

# Load small model
flan_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",   # change to flan-t5-small for even faster
    max_length=512
)

llm = HuggingFacePipeline(pipeline=flan_pipeline)


config.json: 0.00B [00:00, ?B/s]

c:\Users\saksh\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saksh\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP dow

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
C:\Users\saksh\AppData\Local\Temp\ipykernel_17576\3607902014.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=flan_pipeline)


**Retrieval Q&A**

In [27]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type="stuff"
)

# Test
query = "Give me a short summary of the document"
print(qa.run(query))


C:\Users\saksh\AppData\Local\Temp\ipykernel_17576\1543129906.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa.run(query))


Project Neptune: Phase One


**Chat Loop**

In [28]:
while True:
    q = input("Ask a question (or 'exit'): ")
    if q.lower() == "exit":
        break
    print("Answer:", qa.run(q))


Answer: Alice Johnson
Answer: September 1, 2025
Answer: design the user interface for the feedback system
Answer: Develop the core data processing module. - Integrate with the existing analytics dashboard. - Create a user feedback collection system.
